## Google Scholar Crawler
*DISCLAIMER: after retrieving 490 URLs, Google BLOCKED the IP address

*TODO: add selenium and add a manual step for opening a url and showing that we are NOT a robot

In [5]:
keyword_raw = "booter"

In [6]:
import cfscrape
from lxml import etree
import pandas as pd
import urllib
import re
from random import randint
from time import sleep

#Add selenium!!!! 
#change the user agent
#make a small break when google ask for captcha

header = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
    'Cache-Control' : 'max-age=0',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'
    }

In [7]:
page = 1 
df = pd.DataFrame()
    
while True: 
    start_page = (page - 1) * 10
    #################################################################################
    keyword = urllib.parse.quote_plus("\""+keyword_raw+"\"")
    url="https://scholar.google.nl/scholar?start="+str(start_page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(page, url)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url,headers=header).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    #################################################################################
    if len(elements) == 0:
        print("The code stopped for some reason!! Possibly Google blocked us!")
        print("\""+(''.join(html.xpath("//div[@id='infoDiv']/text()")))+"\"")
        break
    #################################################################################
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 1       

1 https://scholar.google.nl/scholar?start=0&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
2 https://scholar.google.nl/scholar?start=10&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
3 https://scholar.google.nl/scholar?start=20&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
4 https://scholar.google.nl/scholar?start=30&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
5 https://scholar.google.nl/scholar?start=40&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
6 https://scholar.google.nl/scholar?start=50&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
7 https://scholar.google.nl/scholar?start=60&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
8 https://scholar.google.nl/scholar?start=70&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
9 https://scholar.google.nl/scholar?start=80&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
10 https://scholar.google.nl/scholar?start=90&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
11 https://scholar.google.nl/scholar?start=100&q=%22booter%22&hl=nl&as_sdt=0,5
sleeping...
12 https://scholar.

## Exporting the output to a csv file

In [8]:
df.to_csv('booter_papers_crawler_google_scholar.csv')

## Further analysis

In [9]:
pd.options.display.max_colwidth = 100
from urllib.parse import urlparse

In [10]:
#Reading again the file in case it is a offline analysis
df = pd.read_csv('booter_papers_crawler_google_scholar.csv',index_col=0).dropna()

In [12]:
len(df)

349

In [38]:
print("TOTAL ENTRIES:",len(df))

TOTAL ENTRIES: 398


In [39]:
source_distribution = df['link'].dropna().apply(lambda x: urlparse(x)[1].replace("www.","")).value_counts()
print("Distinct Source of Information:",len(source_distribution))
print()
print("Frequency of Source of Information:")
print(source_distribution.head(30))

Distinct Source of Information: 80

Frequency of Source of Information:
ieeexplore.ieee.org            74
patents.google.com             51
link.springer.com              40
rfc-editor.org                 34
tools.ietf.org                 21
citeseerx.ist.psu.edu          20
spiedigitallibrary.org          9
pdfs.semanticscholar.org        9
researchgate.net                9
ipsj.ixsq.nii.ac.jp             8
books.google.nl                 8
shiraz.levkowetz.com            8
dbpia.co.kr                     7
dl.acm.org                      6
sciencedirect.com               6
buildbot.tools.ietf.org         5
academia.edu                    5
apps.dtic.mil                   3
onlinelibrary.wiley.com         3
datatracker.ietf.org            3
ftp.nara.wide.ad.jp             2
nanog.org                       2
portaldeconteudo.sbc.org.br     2
watersprings.org                2
ftp.isi.edu                     2
degruyter.com                   2
paper.edu.cn                    2
ejournal.o

The most important source of information are: 
- ieeexplore.ieee.org
- dl.acm.org
- link.springer.com
- pdfs.semanticscholar.org
- sciencedirect.com
- researchgate.net

Of course the RFCs of IETF are also very important:
- rfc-editor.org 
- tools.ietf.org
- datatracker.ietf.org

## Exporting information to csv files based on the source of information

In [40]:
df[df['link'].str.contains("ieeexplore.ieee.org")].to_csv("flowspec_ieee.csv")

In [41]:
df[df['link'].str.contains("link.springer.com")].to_csv("flowspec_springer.csv")

In [42]:
df[df['link'].str.contains("pdfs.semanticscholar.org")].to_csv("flowspec_semanticscholar.csv")

In [45]:
df[df['link'].str.contains("dl.acm.org")].to_csv("flowspec_acm.csv")

## Lessons learned:
- remove papers that have in the title or description/abstract the list of words: [rsvp, qos, quality of service, mpls, intserv, diffserv, multicast]